In [29]:
import pandas as pd
import numpy as np
from pathlib import Path
from os import listdir
import statsmodels.api as sm

In [90]:
#path = "/home/harshit/Project/Returns/2017/"
#entropy=pd.read_csv(path+"continuous_entropy.csv")

In [83]:
path = "/home/harshit/Project/2017/"
entropy=pd.read_csv(path+"entropy.csv")

In [93]:
entropy["ent_risk"] = np.exp(entropy["Entropy"])

In [94]:
data_path = "/home/harshit/Project/2017/"              # finding annual returns of all companies
lis = pd.DataFrame(listdir(data_path),columns = ["Symbol"])
df = pd.DataFrame(columns=["Symbol","ann_return"])

for i in (lis["Symbol"]):            
    if i == "renyi_entropy.csv": continue
    if i == "entropy.csv": continue
    data = pd.read_csv(data_path+i)
    name = i.split('.')[0]
    last = data.iloc[-1][name]
    first= data.iloc[0][name]
    ann_ret = (last-first)*100/first
    df2=pd.DataFrame([[name, ann_ret]],columns=["Symbol","ann_return"])
    df = df.append(df2,ignore_index=True) # a frame having annual returns of all firms

combined = pd.merge(df,entropy,on='Symbol')

In [95]:
#Using co-variance to find beta_risk measure
factors = pd.read_csv("/home/harshit/Project/Returns/averages/FourFactors.csv")

eq_path = "/home/harshit/Project/Returns/2017/"
lis = pd.DataFrame(listdir(eq_path),columns = ["Symbol"])    
df = pd.DataFrame(columns=["Symbol","beta_risk"])
for i in (lis["Symbol"]):
    if i == "continuous_renyi_entropy.csv": continue
    if i == "continuous_entropy.csv": continue
    equity = pd.read_csv(eq_path+i)
    name = i.split(".")[0]
    equity.Date = equity.Date.apply(lambda x: x.replace("-",""))
    equity["Date"] = equity["Date"].apply(pd.to_numeric)
    equity=equity.drop([0])
    equity = pd.merge(factors,equity,on="Date")
    equity["R-Rf %"] = equity["Return"] - equity["Rf %"]
#beta risk = cov(rm-rf,r-rf)/var(rm-rf)

    cov = np.sum((equity["R-Rf %"]-np.mean(equity["R-Rf %"]))*((equity["Rm-Rf %"]-np.mean(equity["Rm-Rf %"]))))/len(equity)
    var = np.sum((equity["Rm-Rf %"]-np.mean(equity["Rm-Rf %"]))**2)/len(equity)
    beta_risk = cov/var
    df2 = pd.DataFrame([[name,beta_risk]],columns=["Symbol","beta_risk"])
    df = df.append(df2,ignore_index=True)
all_data = pd.merge(combined,df,on="Symbol")

In [96]:
X1 = all_data[["beta_risk"]]
X1 = sm.add_constant(X1)
y = all_data[["ann_return"]]
model1 = sm.OLS(y,X1,hasconst=True).fit()

In [97]:
X2 = all_data[["ent_risk"]]
X2 = sm.add_constant(X2)
y = all_data[["ann_return"]]
model2 = sm.OLS(y,X2,hasconst=True).fit()

In [98]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             ann_return   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.365
Date:                Sat, 06 Oct 2018   Prob (F-statistic):              0.124
Time:                        02:40:41   Log-Likelihood:                -6719.1
No. Observations:                1133   AIC:                         1.344e+04
Df Residuals:                    1131   BIC:                         1.345e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         53.4834      3.142     17.025      0.000      47.320      59.647
ent_risk       0.2563      0.167      1.538      0.124      -0.071       0.583
==============================================================================
Omnibus:                     1274.236   Durbin-Watson:                   2.092
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           172880.646
Skew:                           5.328   Prob(JB):                         0.00
Kurtosis:                      62.570   Cond. No.                         21.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""